In [1]:
import os
from Latex.Latex import Latex
import numpy as np
from os import listdir
from skimage import io
import matplotlib.pyplot as plt
from IPython.display import display, Math
import re
import json

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_images = np.load("train_images.npy")
train_labels = np.load("train_labels.npy")

In [3]:
mean_train = np.mean(train_images, axis=0)
std_train = np.std(train_images, axis=0)

In [4]:
model = Latex("model", mean_train, std_train, plotting=False)

ValueError: std_train needs to be defined

In [ ]:
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ']

In [ ]:
files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("formulas/train")) for f in fn]
nlabels = 23
nof_sequences = len(files)
nclasses = nlabels+4+2+1 # 1 for pad and 4 for relative pos, 2 for abs pos and shift from last and width
noclasses = len(ltokens)
isequence = np.zeros((nof_sequences,30,nclasses))
isequence[:,:,-1] = 1 # default all pad
osequence = (noclasses-1)*np.ones((nof_sequences,30), dtype=int)
osequence[:,0] = noclasses # all <GO> in beginning
for i in range(nof_sequences):
    if i % 10 == 0:
        print("Start i: %d" % i)
    formula = io.imread(files[i])
    pos = files[i].rfind("/")
    filename = files[i][pos+1:]
    height, width = formula.shape
    correct = model.filename2formula(filename)
    oseq = model.filename2seq(filename)
    osequence[i,1:len(oseq)+1] = oseq
    latex = model.predict(formula)
    
    last_xmax = 0
    last_ymin = latex['data'][0]['ymin']
    step_c = -1
    for step in latex['data']:
        step_c += 1
        isequence[i][step_c][:nlabels] = step['probs']
        isequence[i][step_c][-1] = 0 # remove pad
        isequence[i][step_c][-7] = step['xmin']/width
        isequence[i][step_c][-6] = step['ymin']/height
        isequence[i][step_c][-5] = (step['xmin']-last_xmax)/10
        last_xmax = step['xmax']
        isequence[i][step_c][-4] = (step['xmax']-step['xmin'])/48
        isequence[i][step_c][-3] = (step['ymin']-last_ymin)/10
        isequence[i][step_c][-2] = (step['ymax']-step['ymin'])/48
        last_ymin = step['ymin']

In [ ]:
np.save("iseq_n", isequence)
np.save("oseq_n", osequence)
with open('files.json', 'w') as f:
    f.write(json.dumps(files[:nof_sequences]))

In [ ]:
iseq = np.zeros((60000,30,nclasses))
osequence = (noclasses-1)*np.ones((60000,30), dtype=int)

In [ ]:
iseq1 = np.load("iseq_n-0-10000.npy")
iseq2 = np.load("iseq_n-10000-20000.npy")
iseq3 = np.load("iseq_n-20000-30000.npy")
iseq4 = np.load("iseq_n-30000-40000.npy")
iseq5 = np.load("iseq_n-40000-50000.npy")
iseq6 = np.load("iseq_n-50000-60000.npy")

In [ ]:
oseq1 = np.load("oseq_n-0-10000.npy")
oseq2 = np.load("oseq_n-10000-20000.npy")
oseq3 = np.load("oseq_n-20000-30000.npy")
oseq4 = np.load("oseq_n-30000-40000.npy")
oseq5 = np.load("oseq_n-40000-50000.npy")
oseq6 = np.load("oseq_n-50000-60000.npy")

In [ ]:
iseq[:10000] = iseq1
iseq[10000:20000] = iseq2
iseq[20000:30000] = iseq3
iseq[30000:40000] = iseq4
iseq[40000:50000] = iseq5
iseq[50000:60000] = iseq6

oseq[:10000] = oseq1
oseq[10000:20000] = oseq2
oseq[20000:30000] = oseq3
oseq[30000:40000] = oseq4
oseq[40000:50000] = oseq5
oseq[50000:60000] = oseq6

np.save("iseq_n", iseq)
np.save("oseq_n", oseq)